# SP API Examples
This notebook is an bare-bone example on how to query Amazon SP API.
Useful for API exploration and debugging purposes.

In [2]:
import boto3
import requests
import json
import os
import zlib
import json
import io

from botocore.exceptions import ClientError

## Configuration

In [3]:
aws_access_key_id = "your_access_key"
aws_secret_access_key = "your_secret_key"
aws_session_token = "your_session_token"

# config this
account_name = "boba"
account_region = "NA"
asin = "B005SP5TX0"
marketplace_id = (
    "ATVPDKIKX0DER"  # developer-docs.amazon.com/sp-api/docs/marketplace-ids
)
base_url = "https://sellingpartnerapi-na.amazon.com"

account_secret_key = "prod/airbyte/sources/sp_api/" + account_name

## Authentication

In [4]:
def get_secret_by_name(secret_name: str):
    secret_str = _get_secret(secret_name)
    return json.loads(secret_str)


def _get_secret(secret_name: str, region_name: str = "eu-west-1"):

    # Create a Secrets Manager client
    session = boto3.session.Session(profile_name="data-prod")
    client = session.client(service_name="secretsmanager", region_name=region_name)

    try:
        get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    except ClientError as e:
        # For a list of exceptions thrown, see
        # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
        raise e

    # Decrypts secret using the associated KMS key.
    return get_secret_value_response["SecretString"]

In [5]:
secret = get_secret_by_name(account_secret_key)

In [ ]:
# Get access token
url = "https://api.amazon.com/auth/o2/token"
client_secret = secret["lwa_client_secret"]
refresh_token = secret["refresh_tokens"][account_region]
client_id = secret["lwa_app_id"]

payload = f"grant_type=refresh_token&refresh_token={refresh_token}&client_id={client_id}&client_secret={client_secret}"
headers = {"Content-Type": "application/x-www-form-urlencoded"}

response = requests.request("POST", url, headers=headers, data=payload)
print(response.json())

access_token = response.json()["access_token"]
print(access_token)

## get Catalog Item

In [ ]:
# Create report

url = f"{base_url}/catalog/2020-12-01/items"

# catalog/2020-12-01/items/:asin?marketplaceIds=boba

payload = {
    "marketplaceIds": [marketplace_id],
}

# payload={
#     "asin": asin,
#     "marketplaceIds":[marketplace_id],
# }

headers = {
    "Accept": "application/json",
    "x-amz-access-token": access_token,
    "X-Amz-Security-Token": aws_session_token,
    "content-type": "application/json; charset=utf-8",
}

response = requests.request("GET", url, headers=headers, data=json.dumps(payload))

response

# response.json()

In [ ]:
## get the report
url = f"{base_url}/reports/2021-06-30/reports/{response.json()['reportId']}"

headers = {
    "Accept": "application/json",
    "x-amz-access-token": access_token,
    "X-Amz-Security-Token": aws_session_token,
    "content-type": "application/json; charset=utf-8",
}

response = requests.request("get", url, headers=headers)
report = response.json()
report

In [62]:
## get the doc of the report
report_doc_id = report["reportDocumentId"]
url = f"{base_url}/reports/2021-06-30/documents/{report_doc_id}"

headers = {
    "Accept": "application/json",
    "x-amz-access-token": access_token,
    "X-Amz-Security-Token": credentials["SessionToken"],
    "content-type": "application/json; charset=utf-8",
}

response = requests.request("get", url, headers=headers)

In [ ]:
response.json()

In [ ]:
url = response.json()["url"]
url

In [65]:
report = requests.get(url)
decoded = zlib.decompress(bytearray(report.content), 15 + 32).decode("iso-8859-1")

In [ ]:
data = json.loads(decoded)
json.dumps(data)

In [67]:
with open(
    f"report_samples/{account_name}_{account_region}_{reportType}.json",
    "w",
    encoding="utf-8",
) as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

## GET_COUPON_PERFORMANCE_REPORT

In [ ]:
# Create report
data_start = "2023-01-01T00:00:00Z"
data_end = "2023-08-31T23:59:59Z"
reportType = "GET_COUPON_PERFORMANCE_REPORT"
# config this url!
url = f"{base_url}/reports/2021-06-30/reports"

payload = {
    "marketplaceIds": [marketplace_id],
    "reportType": reportType,
    "reportOptions": {"couponStartDateFrom": data_start, "couponStartDateTo": data_end},
}

headers = {
    "Accept": "application/json",
    "x-amz-access-token": access_token,
    "X-Amz-Security-Token": credentials["SessionToken"],
    "content-type": "application/json; charset=utf-8",
}

response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response.json()

In [ ]:
## get the report
# config this url!
url = f"{base_url}/reports/2021-06-30/reports/{response.json()['reportId']}"

headers = {
    "Accept": "application/json",
    "x-amz-access-token": access_token,
    "X-Amz-Security-Token": credentials["SessionToken"],
    "content-type": "application/json; charset=utf-8",
}

response = requests.request("get", url, headers=headers)
report = response.json()
report

In [49]:
## get the doc of the report
report_doc_id = report["reportDocumentId"]
url = f"{base_url}/reports/2021-06-30/documents/{report_doc_id}"

headers = {
    "Accept": "application/json",
    "x-amz-access-token": access_token,
    "X-Amz-Security-Token": credentials["SessionToken"],
    "content-type": "application/json; charset=utf-8",
}


response = requests.request("get", url, headers=headers)

In [ ]:
response.json()

In [ ]:
url = response.json()["url"]
url

In [52]:
report = requests.get(url)
decoded = zlib.decompress(bytearray(report.content), 15 + 32).decode("iso-8859-1")

In [ ]:
data = json.loads(decoded)
json.dumps(data)

In [54]:
with open(
    f"report_samples/{account_name}_{account_region}_{reportType}.json",
    "w",
    encoding="utf-8",
) as f:
    json.dump(data, f, ensure_ascii=False, indent=4)